In [1]:
from reader_utils.corpus_builder_utils import *
from note_utils.pitch_dictionary import PitchDictionary
from reader_utils.dataset_builder import DatasetBuilder
from model_utils.model_utils import *
import io

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
pd = PitchDictionary("dataset_objects/pitches_dict.txt")

vocab_size = pd.get_vocabulary_size()
window_size = 4
num_ns = 5
embedding_dim = 32
embedding_layer_name = "n2v_embedding"

db = DatasetBuilder("dataset_objects/full_corpus")

In [3]:
dataset = db.build_word_to_vec_dataset(
        vocab_size, window_size, num_ns, skip_amount=10)

  0%|          | 0/220 [00:00<?, ?it/s]2021-10-12 15:35:30.691236: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-10-12 15:35:30.691393: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1


100%|██████████| 220/220 [09:07<00:00,  2.49s/it]


In [4]:
n2v = build_note_to_vec_model(
        vocab_size, embedding_dim, num_ns, embedding_layer_name)

In [5]:
n2v.fit(dataset, epochs=20)

Epoch 1/20
 17/709 [..............................] - ETA: 4s - loss: 1.7867 - accuracy: 0.4360

2021-10-12 15:45:35.583820: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-10-12 15:45:35.585072: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-10-12 15:45:35.585136: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


709/709 [==============================] - 5s 6ms/step - loss: 0.6982 - accuracy: 0.8333
Epoch 2/20
709/709 [==============================] - 4s 6ms/step - loss: 0.3674 - accuracy: 0.8687
Epoch 3/20
709/709 [==============================] - 5s 7ms/step - loss: 0.3311 - accuracy: 0.8816
Epoch 4/20
709/709 [==============================] - 5s 7ms/step - loss: 0.3182 - accuracy: 0.8863
Epoch 5/20
709/709 [==============================] - 5s 6ms/step - loss: 0.3086 - accuracy: 0.8896
Epoch 6/20
709/709 [==============================] - 5s 6ms/step - loss: 0.3008 - accuracy: 0.8923
Epoch 7/20
709/709 [==============================] - 5s 7ms/step - loss: 0.2956 - accuracy: 0.8946
Epoch 8/20
709/709 [==============================] - 4s 6ms/step - loss: 0.2921 - accuracy: 0.8959
Epoch 9/20
709/709 [==============================] - 5s 7ms/step - loss: 0.2895 - accuracy: 0.8967
Epoch 10/20
709/709 [==============================] - 5s 6ms/step - loss: 0.2876 - accuracy: 0.8972
Epoch 11/2

In [6]:
vocab = pd.get_vocabulary()
weights = n2v.get_layer(embedding_layer_name).get_weights()[0]

In [7]:
vectors_file = io.open('vectors.tsv', 'w', encoding='utf-8')
metadata_file = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
    if index == 0: # skip <unk>
        continue
    vec = weights[index]
    vectors_file.write('\t'.join([str(x) for x in vec]) + "\n")
    metadata_file.write(word + "\n")

vectors_file.close()
metadata_file.close()